In [48]:
import pandas as pd
import numpy as np
from globals import BASE_DIR
import os

available_datasets = ["brightkite", "foursquaretky", "gowalla", "yelp"]
dataset = "brightkite"


In [49]:
def format_best_second(row, col_group, best_criteria):
    """Format the best values as bold and the second-best as underlined, handling ties."""
    values = row[col_group]
    
    if best_criteria == "highest":  # For ndcg
        sorted_indices = np.argsort(values)[::-1]  # Descending
    elif best_criteria == "lowest":  # For arp
        sorted_indices = np.argsort(values)  # Ascending
    elif best_criteria == "closest_to_zero":  # For poplift
        sorted_indices = np.argsort(np.abs(values))  # Closest to zero

    formatted = values.astype(str)  # Convert to strings for LaTeX formatting

    if len(sorted_indices) > 0:
        # Identify the highest value and all its ties
        best_value = values[sorted_indices[0]]
        best_indices = np.where(values == best_value)[0]

        # Bold all highest values
        for idx in best_indices:
            formatted[idx] = f"\\textbf{{{formatted[idx]}}}"

        # Identify the second-highest value, if applicable
        if len(best_indices) == 1 and len(sorted_indices) > 1:
            second_value = values[sorted_indices[np.where(values[sorted_indices] != best_value)[0][0]]]
            second_indices = np.where(values == second_value)[0]

            # Underline all second-highest values
            for idx in second_indices:
                formatted[idx] = f"\\underline{{{formatted[idx]}}}"

    return formatted



def format_best_second_js(row, method_cols):
    for method_col in method_cols:
        values = row[method_col]
        min_value_idx = values.idxmin()  # Get the index of the minimum value
        second_min_value_idx = values.nsmallest(2).idxmax()  # Get the index of the second smallest value

        # Apply bold and underline formatting
        row[method_col] = row[method_col].apply(
            lambda x: f"\\textbf{{{x:.4f}}}" if x == min_value_idx 
            else (f"\\underline{{{x:.4f}}}" if x == second_min_value_idx 
                  else f"{x:.4f}")
        )
    
    return row




In [50]:
df = pd.read_csv(f"{BASE_DIR}/{dataset}_dataset/evaluation_results.csv")
    


In [51]:
df = df.loc[df["model"].isin (["BPR", "LORE", "USG"])]
df = df.loc[df["method"] != "upd"]
df.drop(columns=["js"], inplace=True)



In [52]:
df

,dataset,model,method,user_group,ndcg,arp,poplift,gini
0,brightkite,BPR,baseline,high,0.102547,0.057253,0.132808,0.545449
1,brightkite,BPR,baseline,medium,0.085653,0.050706,0.403658,0.593760
2,brightkite,BPR,baseline,low,0.072833,0.043878,0.854836,0.508060
3,brightkite,BPR,baseline,all,0.086468,0.050650,0.439724,0.640966
4,brightkite,BPR,cp,high,0.102547,0.057253,0.132808,0.545449
5,brightkite,BPR,cp,medium,0.085653,0.050706,0.403658,0.593760
6,brightkite,BPR,cp,low,0.072190,0.039783,0.673434,0.473532
7,brightkite,BPR,cp,all,0.086339,0.049831,0.403443,0.633830
8,brightkite,BPR,cp_min_js,high,0.091190,0.047613,-0.064195,0.525445
9,brightkite,BPR,cp_min_js,medium,0.073668,0.039659,0.091173,0.542263


In [ ]:
import pandas as pd

all_results = []
dataset_latex_parts = []

for dataset in available_datasets:
    df = pd.read_csv(f"{BASE_DIR}/{dataset}_dataset/evaluation_results.csv")
    
    df = df.loc[df["user_group"] == "all"]
    
    # Round float columns to 4 decimals
    float_columns = df.select_dtypes(include="float").columns
    df[float_columns] = df[float_columns].round(4)

    # Pivot table for Gini
    df_pivot_gini = df.pivot(
        index=["model", "user_group"],
        columns="method",
        values=["gini"]
    )

    # Apply formatting to highlight best/second-best values
    method_cols = [col for col in df_pivot_gini.columns if col[0] == "gini"]
    df_pivot_gini[method_cols] = df_pivot_gini.apply(
        lambda row: format_best_second(row, method_cols, "lowest"), axis=1
    )

    # Reset index for proper formatting
    df_reset_gini = df_pivot_gini.reset_index()

    # Convert DataFrame to LaTeX (without table headers, since we merge all)
    gini_latex_part = df_reset_gini.to_latex(
        escape=False,
        index=False,
        header=False,  # No headers inside merged sections
        multicolumn=True,
        multirow=True,
        column_format="llcccc"
    )

    # Add dataset separator before this section
    dataset_separator = f"\\midrule\n\\multicolumn{{6}}{{c}}{{\\textbf{{{dataset.capitalize()}}}}} \\\\"
    dataset_latex_parts.append(dataset_separator + "\n" + gini_latex_part)

# Combine all LaTeX table parts
gini_latex = "\n".join(dataset_latex_parts)

# Add table header and formatting
gini_latex = (
    "\\begin{table}[h]\n"
    "\\centering\n"
    "\\caption{Gini Coefficient Evaluation}\n"
    "\\begin{tabular}{llcccc}\n"
    "\\toprule\n"
    "\\multicolumn{2}{c}{} & \\multicolumn{4}{c}{\\textbf{Gini Coefficient}} \\\\ \\cmidrule(lr){3-6}\n"
    "Model & User Group &  & Method2 & Method3 & Method4 \\\\\n"
    "\\midrule\n"
    + gini_latex +
    "\\bottomrule\n"
    "\\end{tabular}\n"
    "\\end{table}"
)

# Save the LaTeX table
with open(f"{BASE_DIR}/gini_evaluation_combined.tex", "w") as f:
    f.write(gini_latex)

print("Gini table saved as a single LaTeX table with dataset separators.")


Gini table saved as a single LaTeX table.


/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_5869/2307932531.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  best_value = values[sorted_indices[0]]
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_5869/2307932531.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  formatted[idx] = f"\\textbf{{{formatted[idx]}}}"
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_5869/2307932531.py:21: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by

In [45]:
for dataset in available_datasets:
    df = pd.read_csv(f"{BASE_DIR}/{dataset}_dataset/evaluation_results.csv")
    
    df = df.loc[df["user_group"]== "all"]
    
    # Replace user group names with shorter labels
    #df["user_group"] = df["user_group"].apply(lambda x: x.replace("high", "g1").replace("medium", "g2").replace("low", "g3"))

    # Round float columns to 4 decimals
    float_columns = df.select_dtypes(include="float").columns
    df[float_columns] = df[float_columns].round(4)

    # Create Gini pivot table
    df_pivot_gini = df.pivot(
        index=["model", "user_group"],
        columns="method",
        values=["gini"]
    )

    # Apply formatting function
    method_cols = [col for col in df_pivot_gini.columns if col[0] == "gini"]
    df_pivot_gini[method_cols] = df_pivot_gini.apply(
        lambda row: format_best_second(row, method_cols, "lowest"), axis=1
    )

    # Reset index to include model and user group
    df_reset_gini = df_pivot_gini.reset_index()

    # Convert DataFrame to LaTeX
    gini_latex = df_reset_gini.to_latex(
        escape=False,  # Allow LaTeX formatting
        index=False,   # Don't include DataFrame index
        multicolumn=True,
        multirow=True,
        column_format="llcccc",  # Adjust columns
        header=True
    )

    # Add custom LaTeX formatting
    gini_latex = gini_latex.replace(
        "\\toprule", 
        "\\toprule\n\\multicolumn{2}{c}{} & \\multicolumn{4}{c}{\\textbf{Gini Coefficient}} \\ \\cmidrule(lr){3-6}"
    )
    gini_latex = gini_latex.replace("\\midrule", "\\hline")
    gini_latex = gini_latex.replace("\\bottomrule", "\\hline")

    # Save the LaTeX table
    with open(f"{BASE_DIR}/{dataset}_dataset/{dataset}_gini_eval.tex", "w") as f:
        f.write(gini_latex)

    print(f"Gini table saved for {dataset}")


Gini table saved for brightkite
Gini table saved for foursquaretky
Gini table saved for gowalla
Gini table saved for yelp


/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_5869/2307932531.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  best_value = values[sorted_indices[0]]
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_5869/2307932531.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  formatted[idx] = f"\\textbf{{{formatted[idx]}}}"
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_5869/2307932531.py:21: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by

In [62]:
for dataset in available_datasets:
    df = pd.read_csv(f"{BASE_DIR}/{dataset}_dataset/evaluation_results.csv")

    df = df.loc[df["model"].isin (["BPR", "LORE", "USG"])]
    df = df.loc[df["method"] != "upd"]
    df.drop(columns=["js"], inplace=True)
    
    df["user_group"] = df["user_group"].apply(lambda x: x.replace("high", "g1").replace("medium", "g2").replace("low", "g3"))

    float_columns = df.select_dtypes(include="float").columns
    df[float_columns] = df[float_columns].round(4)



    df_pivot = df.pivot(
    index=["model", "user_group"],
    columns="method",
    values=["ndcg", "arp", "poplift", "gini"]
    )

    # Apply formatting for each metric group
    metric_groups = ["ndcg", "arp", "poplift", "gini"]
    criteria = {"ndcg": "highest", "arp": "lowest", "poplift": "closest_to_zero", "gini": "lowest"}

    for metric in metric_groups:
        method_cols = [col for col in df_pivot.columns if col[0] == metric]
        df_pivot[method_cols] = df_pivot.apply(
            lambda row: format_best_second(row, method_cols, criteria[metric]), axis=1
        )


    # Reset the index to include 'model' and 'user_group' in the DataFrame
    df_reset = df_pivot.reset_index()

    # Export to LaTeX while keeping multicolumn and multirow formatting
    latex = df_reset.to_latex(
        escape=False,  # To allow LaTeX formatting
        index=False,   # Don't write the default DataFrame index
        multicolumn=True,
        multirow=True,
        column_format="llcccccccccccc",  # Adjust for the number of columns
        header=True
    )

    # Modify LaTeX for custom formatting
    latex = latex.replace(
        "\\toprule", 
        "\\toprule\n\\multicolumn{2}{c}{} & \\multicolumn{3}{c}{\\textbf{ndcg}} & \\multicolumn{3}{c}{\\textbf{arp}} & \\multicolumn{3}{c}{\\textbf{poplift}} & \\multicolumn{3}{c}{\\textbf{gini}} \\\\ \\cmidrule(lr){3-5} \\cmidrule(lr){6-8} \\cmidrule(lr){9-11} \\cmidrule(lr){12-14}"
    )
    latex = latex.replace("\\midrule", "\\hline")
    latex = latex.replace("\\bottomrule", "\\hline")

    # Save

    with open(f"{BASE_DIR}/{dataset}_dataset/{dataset}_eval_test_if_okay.tex", "w") as f:
        f.write(latex)

    
   

    

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_5869/2307932531.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  best_value = values[sorted_indices[0]]
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_5869/2307932531.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  formatted[idx] = f"\\textbf{{{formatted[idx]}}}"
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_5869/2307932531.py:21: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_5869/2188394089.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_js[float_columns] = df_js[float_columns].round(4)
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_5869/2188394089.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_js[float_columns] = df_js[float_columns].round(4)
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_5869/2188394089.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [38]:
df_pivot_js

js                          
method             baseline      cp cp_min_js     upd
model   user_group                                   
BPR     all          0.2545  0.1248    0.0042  0.0943
        g1           0.1591  0.1567    0.0106  0.1622
        g2           0.3105  0.2990    0.0070  0.1670
        g3           0.2660  0.0008    0.0008  0.0459
LORE    all          0.0263  0.0019    0.0004  0.0871
        g1           0.0769  0.0010    0.0010  0.1622
        g2           0.0253  0.0007    0.0007  0.1449
        g3           0.0114  0.0115    0.0003  0.0489
SimpleX all          0.2081  0.1140    0.0003  0.0940
        g1           0.1555  0.1545    0.0005  0.1622
        g2           0.2620  0.2578    0.0006  0.1652
        g3           0.1871  0.0002    0.0002  0.0479
USG     all          0.2158  0.1177    0.0002  0.0930
        g1           0.1605  0.1445    0.0013  0.1622
        g2           0.2842  0.2763    0.0004  0.1630
        g3           0.1819  0.0002    0.0002  0.0468

In [ ]:
# for dataset in available_datasets:
#     df = pd.read_csv(f"{BASE_DIR}/{dataset}_dataset/evaluation_results.csv")
    
#     df["user_group"] = df["user_group"].apply(lambda x: x.replace("high", "g1").replace("medium", "g2").replace("low", "g3"))

#     float_columns = df.select_dtypes(include="float").columns
#     df[float_columns] = df[float_columns].round(4)


#     df_pivot = df.pivot(
#     index=["model", "user_group"],
#     columns="method",
#     values=["ndcg", "arp", "poplift"]
#     )

#     # Apply formatting for each metric group
#     metric_groups = ["ndcg", "arp", "poplift"]
#     criteria = {"ndcg": "highest", "arp": "lowest", "poplift": "closest_to_zero"}

#     for metric in metric_groups:
#         method_cols = [col for col in df_pivot.columns if col[0] == metric]
#         df_pivot[method_cols] = df_pivot.apply(
#             lambda row: format_best_second(row, method_cols, criteria[metric]), axis=1
#         )


#     df_pivot_js = df.pivot(
#     index=["model", "user_group"],
#     columns="method",
#     values=["ndcg", "arp", "poplift", "js"]
#     )
#     # ----- Extra Table for JS -----
#     # Filter JS columns
#     js_cols = [col for col in df_pivot_js.columns if col[0] == "js"]
#     js_table = df_pivot_js[js_cols].reset_index()

#     # Apply formatting for JS (highlight the lowest value)
#     js_table[js_cols] = js_table.apply(
#         lambda row: format_best_second(row, js_cols, criteria["js"]), axis=1
#     )

#     # Export JS-only table to LaTeX
#     js_latex = js_table.to_latex(
#         escape=False,
#         index=False,
#         multicolumn=True,
#         multirow=True,
#         column_format="llcccc",  # Adjust for JS columns
#         header=True
#     )

#     # Custom LaTeX formatting for JS table
#     js_latex = js_latex.replace(
#         "\\toprule", 
#         "\\toprule\n\\multicolumn{2}{c}{} & \\multicolumn{4}{c}{\\textbf{Jensen-Shannon Divergence (JSD)}}\\\\ \\cmidrule(lr){3-6}"
#     )
#     js_latex = js_latex.replace("\\midrule", "\\hline")
#     js_latex = js_latex.replace("\\bottomrule", "\\hline")

#     # Save JS-only table
#     with open(f"{BASE_DIR}/{dataset}_dataset/{dataset}_js_eval.tex", "w") as f:
#         f.write(js_latex)

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_14429/2000175152.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  best_value = values[sorted_indices[0]]
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_14429/2000175152.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  formatted[idx] = f"\\textbf{{{formatted[idx]}}}"
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_14429/2000175152.py:21: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value